<h1 align='center'>  Classification of Multiple Classes. </h1>

<p> &nbsp;&nbsp; Sometimes Neural Networks have to predict multiple classes, More then one class solution. There are 2 ways to solve this problem 
<ol>
    <li> Multi Label Classification </li>
    <li> Multi Class Classification </li>
</ol>

<p>&nbsp; 1. Multi Label Classification </br>
&nbsp;&nbsp;&nbsp; In this approach we basically have multiple outputs for multiple classes and get output as 1 or 0 for each neurons output. It invovles one hot encoding process of the dataset </p>

<p>&nbsp; 2. Multi Class Classification </br>
&nbsp;&nbsp;&nbsp; In this approach we use just the single output neuron to predict different classess itself. This involves label encoding process of dataset </p>

<img width="1200" height="800" src='https://www.section.io/engineering-education/multi-label-classification-with-scikit-multilearn/label-powerset.png'>

<p> Let's Work on Multi Class Classification first after which we can move to multi label classification </br>
We Can work with the <b><i> IRIS FLOWER DATASET </i></b>

<h5 align='center'> Dataset: </h5>
<img src='https://miro.medium.com/max/638/0*2c7voFri9cIXGrc4'>

<h3 align='center'> 0. Installing Dependencies </h3>

In [1]:
!pip install tensorflow
!pip install numpy 
!pip install pandas 
!pip install -U scikit-learn

<h3 align='center'> 1. Importing Dependenceis </h3>

<p> We are going to import all the dependencies for this project. The Dependencies are: </p>
<ul>
    <li> Pandas </li>
    <li> Numpy </li>
    <li>tensorflow </li>
    <li> sklearn </li>
</ul>

In [2]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from sklearn.model_selection import train_test_split

<h3 align='center'> 2. Handling Dataset </h3>
<p> &nbsp;&nbsp;&nbsp; We are going to handle the dataset in following ways: </p>
<ul>
    <li> Importing Dataset </li>
    <li> Label encoding </li>
    <li> One hot encoding </li>
</ul>

<h4> Importing Dataset </h4>

In [3]:
def read_dataset(path):
    data=pd.read_csv(path)
    return data 

def get_class_dict(dataset):
    class_list=list(set(dataset['species'].to_list()))
    class_dct={}
    for i in range(len(class_list)):
        class_dct[class_list[i]]=i
    return class_dct

dataset=read_dataset('IRIS.csv')
classes_dict=get_class_dict(dataset)
#Creating copies of dataset so it can be used to obtain 2 values.
d_temp1=dataset.copy()
d_temp2=dataset.copy()
del dataset #Deleting Dataset freeing some space


<h4> Label Encodng </h4>
<p>&nbsp; &nbsp; &nbsp; We will have each class value in dictionary as label for the class this method of getting out is called label encoding we will use it for multi class classification </p>

In [4]:
def label_encoding_dataset(d_temp1,class_dict):
    for i in class_dict.keys():
        d_temp1.loc[d_temp1['species']==i,'species']=class_dict[i]
    
    return d_temp1        
print(classes_dict)
dataset_label_encoded=label_encoding_dataset(d_temp1,classes_dict)
dataset_label_encoded



{'Iris-versicolor': 0, 'Iris-virginica': 1, 'Iris-setosa': 2}


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,2
1,4.9,3.0,1.4,0.2,2
2,4.7,3.2,1.3,0.2,2
3,4.6,3.1,1.5,0.2,2
4,5.0,3.6,1.4,0.2,2
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,1
148,6.2,3.4,5.4,2.3,1


<h4> One Hot Encoding <h4>
<p>&nbsp; &nbsp; &nbsp; Now we are going to have 3 labels each will have 1 or 0 present on it This approach will give us 3 outputs and each output will have 0 or 1 present on it demonstrating yes or no of the output this technique is called one hot encoding and this classification is called Multi Label classification 

In [5]:
def one_hot_encoding_dataset(dataset,classes_dict):
    def encode(x,i):
        if x==i:
            return 1
        else: 
            return 0

    def add_columns(dataset,classes_dict):
        dataset_final=dataset.drop(columns=['species'])
        for i in classes_dict.keys():
            dataset_final[i]=dataset['species'].apply(encode,args =(i,))
        return dataset_final


    dataset_final=add_columns(dataset,classes_dict)
    return dataset_final

dataset_one_hot_encoded=one_hot_encoding_dataset(d_temp2,classes_dict)
dataset_one_hot_encoded

,sepal_length,sepal_width,petal_length,petal_width,Iris-versicolor,Iris-virginica,Iris-setosa
0,5.1,3.5,1.4,0.2,0,0,1
1,4.9,3.0,1.4,0.2,0,0,1
2,4.7,3.2,1.3,0.2,0,0,1
3,4.6,3.1,1.5,0.2,0,0,1
4,5.0,3.6,1.4,0.2,0,0,1
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,1,0
146,6.3,2.5,5.0,1.9,0,1,0
147,6.5,3.0,5.2,2.0,0,1,0
148,6.2,3.4,5.4,2.3,0,1,0


<h3 align='center'>3. Multi Class Classification </h3>
<p> &nbsp; &nbsp; &nbsp; Multi Class classification will have single output in the neural network that will work out different output values for output neurons.We are going to use softmax for activation function to get the required output.We can see the following image to learn more </p>
<img src='https://miro.medium.com/max/1400/1*ReYpdIZ3ZSAPb2W8cJpkBg.jpeg'>

</br> Our final output neuron of the neural network will be just one consisting of 3 possible values Making our entire architecture of Neural Network as:
</br></br>
<img src='./Diagrams/Multi Class Classification Architecture.jpg'>

<p> The number of neuron assigned to each layer is arbitary at this point but the final layer should have as many neurons as classes to predict because of how softmax activation function works </p>


In [6]:
def create_model():
    model=tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=9,activation='sigmoid',input_dim=4))
    model.add(tf.keras.layers.Dense(units=18,activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=9,activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=3,activation='softmax'))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),metrics=['accuracy'])
    #model.summary()
    return model

def get_X_Y_label_encoded(dataset):
    X=np.array(dataset.drop(columns=['species'])).astype(np.float32)
    Y=(np.array(dataset['species']).astype(np.float32)).reshape(-1,1)
    return X,Y

multi_class_model=create_model()
X,Y=get_X_Y_label_encoded(dataset_label_encoded)
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.33, random_state=42)
history_mcm=multi_class_model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=500,batch_size=64)



Epoch 1/500
2/2 [==============================] - 1s 138ms/step - loss: 1.1643 - accuracy: 0.3100 - val_loss: 1.1136 - val_accuracy: 0.3800
Epoch 2/500
2/2 [==============================] - 0s 27ms/step - loss: 1.1198 - accuracy: 0.3200 - val_loss: 1.1036 - val_accuracy: 0.3000
Epoch 3/500
2/2 [==============================] - 0s 33ms/step - loss: 1.1012 - accuracy: 0.3500 - val_loss: 1.1096 - val_accuracy: 0.3000
Epoch 4/500
2/2 [==============================] - 0s 29ms/step - loss: 1.0978 - accuracy: 0.4300 - val_loss: 1.1203 - val_accuracy: 0.3200
Epoch 5/500
2/2 [==============================] - 0s 28ms/step - loss: 1.1011 - accuracy: 0.3400 - val_loss: 1.1241 - val_accuracy: 0.3200
Epoch 6/500
2/2 [==============================] - 0s 32ms/step - loss: 1.1070 - accuracy: 0.3400 - val_loss: 1.1227 - val_accuracy: 0.3200
Epoch 7/500
2/2 [==============================] - 0s 33ms/step - loss: 1.1058 - accuracy: 0.3400 - val_loss: 1.1170 - val_accuracy: 0.3200
Epoch 8/500
2/2 [==

2/2 [==============================] - 0s 29ms/step - loss: 0.2663 - accuracy: 0.9700 - val_loss: 0.2271 - val_accuracy: 0.9800
Epoch 60/500
2/2 [==============================] - 0s 27ms/step - loss: 0.2525 - accuracy: 0.9600 - val_loss: 0.2141 - val_accuracy: 0.9800
Epoch 61/500
2/2 [==============================] - 0s 27ms/step - loss: 0.2391 - accuracy: 0.9800 - val_loss: 0.2060 - val_accuracy: 0.9800
Epoch 62/500
2/2 [==============================] - 0s 26ms/step - loss: 0.2253 - accuracy: 0.9800 - val_loss: 0.1908 - val_accuracy: 0.9800
Epoch 63/500
2/2 [==============================] - 0s 26ms/step - loss: 0.2119 - accuracy: 0.9800 - val_loss: 0.1791 - val_accuracy: 0.9800
Epoch 64/500
2/2 [==============================] - 0s 26ms/step - loss: 0.2056 - accuracy: 0.9600 - val_loss: 0.1691 - val_accuracy: 0.9800
Epoch 65/500
2/2 [==============================] - 0s 28ms/step - loss: 0.1914 - accuracy: 0.9700 - val_loss: 0.1735 - val_accuracy: 1.0000
Epoch 66/500
2/2 [========

2/2 [==============================] - 0s 47ms/step - loss: 0.0773 - accuracy: 0.9800 - val_loss: 0.0553 - val_accuracy: 0.9800
Epoch 118/500
2/2 [==============================] - 0s 46ms/step - loss: 0.0688 - accuracy: 0.9800 - val_loss: 0.0503 - val_accuracy: 0.9800
Epoch 119/500
2/2 [==============================] - 0s 47ms/step - loss: 0.0864 - accuracy: 0.9700 - val_loss: 0.0492 - val_accuracy: 0.9800
Epoch 120/500
2/2 [==============================] - 0s 46ms/step - loss: 0.0785 - accuracy: 0.9800 - val_loss: 0.0578 - val_accuracy: 0.9800
Epoch 121/500
2/2 [==============================] - 0s 53ms/step - loss: 0.0654 - accuracy: 0.9900 - val_loss: 0.1028 - val_accuracy: 0.9600
Epoch 122/500
2/2 [==============================] - 0s 49ms/step - loss: 0.0821 - accuracy: 0.9600 - val_loss: 0.0928 - val_accuracy: 0.9600
Epoch 123/500
2/2 [==============================] - 0s 56ms/step - loss: 0.0695 - accuracy: 0.9800 - val_loss: 0.0522 - val_accuracy: 0.9800
Epoch 124/500
2/2 [=

Epoch 175/500
2/2 [==============================] - 0s 54ms/step - loss: 0.0588 - accuracy: 0.9900 - val_loss: 0.0539 - val_accuracy: 0.9800
Epoch 176/500
2/2 [==============================] - 0s 58ms/step - loss: 0.0599 - accuracy: 0.9800 - val_loss: 0.0649 - val_accuracy: 0.9800
Epoch 177/500
2/2 [==============================] - 0s 71ms/step - loss: 0.0565 - accuracy: 0.9800 - val_loss: 0.0516 - val_accuracy: 0.9800
Epoch 178/500
2/2 [==============================] - 0s 69ms/step - loss: 0.0558 - accuracy: 0.9900 - val_loss: 0.0410 - val_accuracy: 0.9800
Epoch 179/500
2/2 [==============================] - 0s 70ms/step - loss: 0.0591 - accuracy: 0.9800 - val_loss: 0.0423 - val_accuracy: 0.9800
Epoch 180/500
2/2 [==============================] - 0s 69ms/step - loss: 0.0578 - accuracy: 0.9800 - val_loss: 0.0528 - val_accuracy: 0.9800
Epoch 181/500
2/2 [==============================] - 0s 107ms/step - loss: 0.0536 - accuracy: 0.9900 - val_loss: 0.0796 - val_accuracy: 0.9600
Epoch

Epoch 233/500
2/2 [==============================] - 0s 50ms/step - loss: 0.0500 - accuracy: 0.9800 - val_loss: 0.0552 - val_accuracy: 0.9800
Epoch 234/500
2/2 [==============================] - 0s 44ms/step - loss: 0.0499 - accuracy: 0.9900 - val_loss: 0.0502 - val_accuracy: 0.9800
Epoch 235/500
2/2 [==============================] - 0s 50ms/step - loss: 0.0516 - accuracy: 0.9900 - val_loss: 0.0514 - val_accuracy: 0.9800
Epoch 236/500
2/2 [==============================] - 0s 52ms/step - loss: 0.0496 - accuracy: 0.9900 - val_loss: 0.0464 - val_accuracy: 0.9800
Epoch 237/500
2/2 [==============================] - 0s 53ms/step - loss: 0.0557 - accuracy: 0.9900 - val_loss: 0.0507 - val_accuracy: 0.9800
Epoch 238/500
2/2 [==============================] - 0s 44ms/step - loss: 0.0483 - accuracy: 0.9800 - val_loss: 0.0780 - val_accuracy: 0.9600
Epoch 239/500
2/2 [==============================] - 0s 50ms/step - loss: 0.0576 - accuracy: 0.9600 - val_loss: 0.0725 - val_accuracy: 0.9800
Epoch 

Epoch 291/500
2/2 [==============================] - 0s 47ms/step - loss: 0.0472 - accuracy: 0.9800 - val_loss: 0.0506 - val_accuracy: 0.9800
Epoch 292/500
2/2 [==============================] - 0s 50ms/step - loss: 0.0540 - accuracy: 0.9900 - val_loss: 0.0339 - val_accuracy: 0.9800
Epoch 293/500
2/2 [==============================] - 0s 46ms/step - loss: 0.0566 - accuracy: 0.9800 - val_loss: 0.0448 - val_accuracy: 0.9800
Epoch 294/500
2/2 [==============================] - 0s 52ms/step - loss: 0.0466 - accuracy: 0.9900 - val_loss: 0.0613 - val_accuracy: 0.9800
Epoch 295/500
2/2 [==============================] - 0s 53ms/step - loss: 0.0470 - accuracy: 0.9800 - val_loss: 0.0799 - val_accuracy: 0.9600
Epoch 296/500
2/2 [==============================] - 0s 75ms/step - loss: 0.0528 - accuracy: 0.9800 - val_loss: 0.0672 - val_accuracy: 0.9800
Epoch 297/500
2/2 [==============================] - 0s 57ms/step - loss: 0.0491 - accuracy: 0.9800 - val_loss: 0.0589 - val_accuracy: 0.9800
Epoch 

Epoch 349/500
2/2 [==============================] - 0s 76ms/step - loss: 0.0429 - accuracy: 0.9900 - val_loss: 0.0440 - val_accuracy: 0.9800
Epoch 350/500
2/2 [==============================] - 0s 64ms/step - loss: 0.0462 - accuracy: 0.9900 - val_loss: 0.0485 - val_accuracy: 0.9800
Epoch 351/500
2/2 [==============================] - 0s 69ms/step - loss: 0.0429 - accuracy: 0.9900 - val_loss: 0.0683 - val_accuracy: 0.9800
Epoch 352/500
2/2 [==============================] - 0s 73ms/step - loss: 0.0423 - accuracy: 0.9800 - val_loss: 0.0858 - val_accuracy: 0.9600
Epoch 353/500
2/2 [==============================] - 0s 69ms/step - loss: 0.0472 - accuracy: 0.9800 - val_loss: 0.0772 - val_accuracy: 0.9800
Epoch 354/500
2/2 [==============================] - 0s 71ms/step - loss: 0.0434 - accuracy: 0.9800 - val_loss: 0.0570 - val_accuracy: 0.9800
Epoch 355/500
2/2 [==============================] - 0s 68ms/step - loss: 0.0428 - accuracy: 0.9900 - val_loss: 0.0444 - val_accuracy: 0.9800
Epoch 

Epoch 407/500
2/2 [==============================] - 0s 79ms/step - loss: 0.0477 - accuracy: 0.9900 - val_loss: 0.0518 - val_accuracy: 0.9800
Epoch 408/500
2/2 [==============================] - 0s 69ms/step - loss: 0.0471 - accuracy: 0.9800 - val_loss: 0.0810 - val_accuracy: 0.9800
Epoch 409/500
2/2 [==============================] - 0s 71ms/step - loss: 0.0428 - accuracy: 0.9800 - val_loss: 0.0735 - val_accuracy: 0.9800
Epoch 410/500
2/2 [==============================] - 0s 57ms/step - loss: 0.0402 - accuracy: 0.9800 - val_loss: 0.0568 - val_accuracy: 0.9800
Epoch 411/500
2/2 [==============================] - 0s 66ms/step - loss: 0.0413 - accuracy: 0.9900 - val_loss: 0.0509 - val_accuracy: 0.9800
Epoch 412/500
2/2 [==============================] - 0s 72ms/step - loss: 0.0420 - accuracy: 0.9900 - val_loss: 0.0600 - val_accuracy: 0.9800
Epoch 413/500
2/2 [==============================] - 0s 96ms/step - loss: 0.0399 - accuracy: 0.9900 - val_loss: 0.0715 - val_accuracy: 0.9800
Epoch 

Epoch 465/500
2/2 [==============================] - 0s 56ms/step - loss: 0.0376 - accuracy: 0.9900 - val_loss: 0.0965 - val_accuracy: 0.9600
Epoch 466/500
2/2 [==============================] - 0s 61ms/step - loss: 0.0547 - accuracy: 0.9800 - val_loss: 0.1308 - val_accuracy: 0.9600
Epoch 467/500
2/2 [==============================] - 0s 60ms/step - loss: 0.0755 - accuracy: 0.9600 - val_loss: 0.0658 - val_accuracy: 0.9800
Epoch 468/500
2/2 [==============================] - 0s 54ms/step - loss: 0.0441 - accuracy: 0.9900 - val_loss: 0.0449 - val_accuracy: 0.9800
Epoch 469/500
2/2 [==============================] - 0s 50ms/step - loss: 0.0447 - accuracy: 0.9900 - val_loss: 0.0575 - val_accuracy: 0.9800
Epoch 470/500
2/2 [==============================] - 0s 54ms/step - loss: 0.0446 - accuracy: 0.9800 - val_loss: 0.0909 - val_accuracy: 0.9600
Epoch 471/500
2/2 [==============================] - 0s 47ms/step - loss: 0.0443 - accuracy: 0.9800 - val_loss: 0.0809 - val_accuracy: 0.9800
Epoch 

<h3 align='center'> 3.1 Testing </h3>

We are going to input new custom dataset to see how well it predicts 

In [7]:
def predict_output(model,input_user):
    output=model.predict([input_user])
    print(output)
    output=np.argmax(output[0])
    output=list({i for i in classes_dict.keys() if classes_dict[i]==output})
    return output[0]

def get_input_from_users(length):
    input_user=[]
    for i in range(1,length+1):
        input_user.append(float(input(f'Enter {i} ipnut ')))
    return input_user
    
input_1=[5.1,3.5,1.4,0.2]
print(predict_output(multi_class_model,input_1))

input_2=get_input_from_users(dataset_label_encoded.shape[1]-1)
print(predict_output(multi_class_model,input_2))


1/1 [==============================] - 0s 190ms/step
[[9.3575410e-04 1.7447752e-05 9.9904674e-01]]
Iris-setosa
Enter 1 ipnut 2.5
Enter 2 ipnut 1.5
Enter 3 ipnut 3.3
Enter 4 ipnut 5.5
1/1 [==============================] - 0s 70ms/step
[[1.296998e-03 9.986941e-01 8.958329e-06]]
Iris-virginica


<h3 align='center'> 4. Multi Label Classification </h3> </br>
 Multi-label classification or multi-output classification is a variant of the classification problem where multiple nonexclusive labels may be assigned to each instance. This classification will give multiple outputs for each class present in the provided input maybe a human and a dog together will be classified as man=1 and dog=1. This image gives perfect summary of the sentences written. 

 <img src='https://www.mathworks.com/help/examples/nnet/win64/MultilabelImageClassificationUsingDeepLearningExample_01.png'>
 </br></br>
 For our test case we will have a neural network architecture as:
 <img src='Diagrams\Multi Label Classification.jpg'>

In [8]:
def create_multi_label_model():
    model=tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=9,activation='sigmoid',input_dim=4))
    model.add(tf.keras.layers.Dense(units=18,activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=9,activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=3,activation='sigmoid'))
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),metrics=['accuracy'])
    #model.summary()
    return model

def get_X_Y_one_hot_encoded(dataset):
    X=np.array(dataset[['sepal_length','sepal_width','petal_length','petal_width']])
    Y=np.array(dataset.drop(columns=['sepal_length','sepal_width','petal_length','petal_width']))
    return X,Y

multi_label_model=create_multi_label_model()
X,Y=get_X_Y_one_hot_encoded(dataset_one_hot_encoded)
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.33, random_state=42)
history_mlm=multi_label_model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=500,batch_size=64)




Epoch 1/500
2/2 [==============================] - 2s 298ms/step - loss: 0.6678 - accuracy: 0.3500 - val_loss: 0.6435 - val_accuracy: 0.3000
Epoch 2/500
2/2 [==============================] - 0s 53ms/step - loss: 0.6445 - accuracy: 0.3500 - val_loss: 0.6364 - val_accuracy: 0.3200
Epoch 3/500
2/2 [==============================] - 0s 56ms/step - loss: 0.6356 - accuracy: 0.3500 - val_loss: 0.6373 - val_accuracy: 0.3200
Epoch 4/500
2/2 [==============================] - 0s 55ms/step - loss: 0.6351 - accuracy: 0.3400 - val_loss: 0.6401 - val_accuracy: 0.3200
Epoch 5/500
2/2 [==============================] - 0s 53ms/step - loss: 0.6370 - accuracy: 0.3400 - val_loss: 0.6419 - val_accuracy: 0.3200
Epoch 6/500
2/2 [==============================] - 0s 52ms/step - loss: 0.6375 - accuracy: 0.3400 - val_loss: 0.6422 - val_accuracy: 0.3200
Epoch 7/500
2/2 [==============================] - 0s 58ms/step - loss: 0.6370 - accuracy: 0.3400 - val_loss: 0.6407 - val_accuracy: 0.3200
Epoch 8/500
2/2 [==

2/2 [==============================] - 0s 52ms/step - loss: 0.2543 - accuracy: 0.9600 - val_loss: 0.2290 - val_accuracy: 0.9800
Epoch 60/500
2/2 [==============================] - 0s 54ms/step - loss: 0.2471 - accuracy: 0.9600 - val_loss: 0.2247 - val_accuracy: 0.9800
Epoch 61/500
2/2 [==============================] - 0s 49ms/step - loss: 0.2411 - accuracy: 0.9600 - val_loss: 0.2193 - val_accuracy: 0.9800
Epoch 62/500
2/2 [==============================] - 0s 50ms/step - loss: 0.2347 - accuracy: 0.9700 - val_loss: 0.2116 - val_accuracy: 0.9800
Epoch 63/500
2/2 [==============================] - 0s 51ms/step - loss: 0.2264 - accuracy: 0.9600 - val_loss: 0.2063 - val_accuracy: 0.9800
Epoch 64/500
2/2 [==============================] - 0s 78ms/step - loss: 0.2236 - accuracy: 0.9600 - val_loss: 0.2002 - val_accuracy: 0.9800
Epoch 65/500
2/2 [==============================] - 0s 63ms/step - loss: 0.2150 - accuracy: 0.9600 - val_loss: 0.1976 - val_accuracy: 0.9800
Epoch 66/500
2/2 [========

2/2 [==============================] - 0s 54ms/step - loss: 0.0740 - accuracy: 0.9900 - val_loss: 0.0661 - val_accuracy: 0.9800
Epoch 118/500
2/2 [==============================] - 0s 63ms/step - loss: 0.0778 - accuracy: 0.9700 - val_loss: 0.0667 - val_accuracy: 0.9600
Epoch 119/500
2/2 [==============================] - 0s 61ms/step - loss: 0.0771 - accuracy: 0.9800 - val_loss: 0.0782 - val_accuracy: 0.9800
Epoch 120/500
2/2 [==============================] - 0s 61ms/step - loss: 0.0720 - accuracy: 0.9800 - val_loss: 0.0745 - val_accuracy: 0.9800
Epoch 121/500
2/2 [==============================] - 0s 55ms/step - loss: 0.0712 - accuracy: 0.9700 - val_loss: 0.0671 - val_accuracy: 0.9800
Epoch 122/500
2/2 [==============================] - 0s 54ms/step - loss: 0.0702 - accuracy: 0.9800 - val_loss: 0.0681 - val_accuracy: 0.9800
Epoch 123/500
2/2 [==============================] - 0s 53ms/step - loss: 0.0716 - accuracy: 0.9800 - val_loss: 0.0709 - val_accuracy: 0.9800
Epoch 124/500
2/2 [=

Epoch 175/500
2/2 [==============================] - 0s 50ms/step - loss: 0.0489 - accuracy: 0.9900 - val_loss: 0.0512 - val_accuracy: 0.9800
Epoch 176/500
2/2 [==============================] - 0s 48ms/step - loss: 0.0490 - accuracy: 0.9900 - val_loss: 0.0555 - val_accuracy: 0.9800
Epoch 177/500
2/2 [==============================] - 0s 51ms/step - loss: 0.0484 - accuracy: 0.9900 - val_loss: 0.0539 - val_accuracy: 0.9800
Epoch 178/500
2/2 [==============================] - 0s 54ms/step - loss: 0.0475 - accuracy: 0.9900 - val_loss: 0.0564 - val_accuracy: 0.9800
Epoch 179/500
2/2 [==============================] - 0s 48ms/step - loss: 0.0476 - accuracy: 0.9900 - val_loss: 0.0563 - val_accuracy: 0.9800
Epoch 180/500
2/2 [==============================] - 0s 51ms/step - loss: 0.0476 - accuracy: 0.9800 - val_loss: 0.0512 - val_accuracy: 0.9800
Epoch 181/500
2/2 [==============================] - 0s 54ms/step - loss: 0.0467 - accuracy: 0.9900 - val_loss: 0.0457 - val_accuracy: 0.9800
Epoch 

Epoch 233/500
2/2 [==============================] - 0s 91ms/step - loss: 0.0420 - accuracy: 0.9700 - val_loss: 0.0453 - val_accuracy: 0.9800
Epoch 234/500
2/2 [==============================] - 0s 70ms/step - loss: 0.0412 - accuracy: 0.9900 - val_loss: 0.0400 - val_accuracy: 0.9800
Epoch 235/500
2/2 [==============================] - 0s 54ms/step - loss: 0.0418 - accuracy: 0.9900 - val_loss: 0.0461 - val_accuracy: 0.9800
Epoch 236/500
2/2 [==============================] - 0s 52ms/step - loss: 0.0390 - accuracy: 0.9900 - val_loss: 0.0533 - val_accuracy: 0.9800
Epoch 237/500
2/2 [==============================] - 0s 63ms/step - loss: 0.0398 - accuracy: 0.9800 - val_loss: 0.0589 - val_accuracy: 0.9800
Epoch 238/500
2/2 [==============================] - 0s 67ms/step - loss: 0.0404 - accuracy: 0.9800 - val_loss: 0.0492 - val_accuracy: 0.9800
Epoch 239/500
2/2 [==============================] - 0s 64ms/step - loss: 0.0385 - accuracy: 0.9900 - val_loss: 0.0430 - val_accuracy: 0.9800
Epoch 

Epoch 291/500
2/2 [==============================] - 0s 53ms/step - loss: 0.0348 - accuracy: 0.9800 - val_loss: 0.0369 - val_accuracy: 0.9800
Epoch 292/500
2/2 [==============================] - 0s 48ms/step - loss: 0.0383 - accuracy: 0.9900 - val_loss: 0.0371 - val_accuracy: 0.9800
Epoch 293/500
2/2 [==============================] - 0s 50ms/step - loss: 0.0362 - accuracy: 0.9900 - val_loss: 0.0502 - val_accuracy: 0.9800
Epoch 294/500
2/2 [==============================] - 0s 49ms/step - loss: 0.0374 - accuracy: 0.9800 - val_loss: 0.0646 - val_accuracy: 0.9600
Epoch 295/500
2/2 [==============================] - 0s 50ms/step - loss: 0.0376 - accuracy: 0.9700 - val_loss: 0.0495 - val_accuracy: 0.9800
Epoch 296/500
2/2 [==============================] - 0s 52ms/step - loss: 0.0345 - accuracy: 0.9900 - val_loss: 0.0382 - val_accuracy: 0.9800
Epoch 297/500
2/2 [==============================] - 0s 51ms/step - loss: 0.0397 - accuracy: 0.9900 - val_loss: 0.0420 - val_accuracy: 0.9800
Epoch 

Epoch 349/500
2/2 [==============================] - 0s 53ms/step - loss: 0.0401 - accuracy: 0.9700 - val_loss: 0.0580 - val_accuracy: 0.9800
Epoch 350/500
2/2 [==============================] - 0s 51ms/step - loss: 0.0303 - accuracy: 0.9800 - val_loss: 0.0339 - val_accuracy: 0.9800
Epoch 351/500
2/2 [==============================] - 0s 47ms/step - loss: 0.0371 - accuracy: 0.9900 - val_loss: 0.0308 - val_accuracy: 0.9800
Epoch 352/500
2/2 [==============================] - 0s 52ms/step - loss: 0.0412 - accuracy: 0.9700 - val_loss: 0.0400 - val_accuracy: 0.9800
Epoch 353/500
2/2 [==============================] - 0s 52ms/step - loss: 0.0315 - accuracy: 0.9900 - val_loss: 0.0606 - val_accuracy: 0.9800
Epoch 354/500
2/2 [==============================] - 0s 53ms/step - loss: 0.0332 - accuracy: 0.9800 - val_loss: 0.0668 - val_accuracy: 0.9600
Epoch 355/500
2/2 [==============================] - 0s 55ms/step - loss: 0.0361 - accuracy: 0.9700 - val_loss: 0.0549 - val_accuracy: 0.9800
Epoch 

Epoch 407/500
2/2 [==============================] - 0s 49ms/step - loss: 0.0344 - accuracy: 0.9700 - val_loss: 0.0515 - val_accuracy: 0.9800
Epoch 408/500
2/2 [==============================] - 0s 52ms/step - loss: 0.0281 - accuracy: 0.9900 - val_loss: 0.0342 - val_accuracy: 0.9800
Epoch 409/500
2/2 [==============================] - 0s 47ms/step - loss: 0.0348 - accuracy: 0.9900 - val_loss: 0.0351 - val_accuracy: 0.9800
Epoch 410/500
2/2 [==============================] - 0s 50ms/step - loss: 0.0354 - accuracy: 0.9900 - val_loss: 0.0504 - val_accuracy: 0.9800
Epoch 411/500
2/2 [==============================] - 0s 48ms/step - loss: 0.0289 - accuracy: 0.9900 - val_loss: 0.0597 - val_accuracy: 0.9800
Epoch 412/500
2/2 [==============================] - 0s 51ms/step - loss: 0.0406 - accuracy: 0.9700 - val_loss: 0.0581 - val_accuracy: 0.9800
Epoch 413/500
2/2 [==============================] - 0s 49ms/step - loss: 0.0262 - accuracy: 0.9900 - val_loss: 0.0365 - val_accuracy: 0.9800
Epoch 

Epoch 465/500
2/2 [==============================] - 0s 48ms/step - loss: 0.0270 - accuracy: 0.9900 - val_loss: 0.0468 - val_accuracy: 0.9800
Epoch 466/500
2/2 [==============================] - 0s 47ms/step - loss: 0.0286 - accuracy: 0.9900 - val_loss: 0.0453 - val_accuracy: 0.9800
Epoch 467/500
2/2 [==============================] - 0s 47ms/step - loss: 0.0295 - accuracy: 0.9900 - val_loss: 0.0517 - val_accuracy: 0.9800
Epoch 468/500
2/2 [==============================] - 0s 47ms/step - loss: 0.0302 - accuracy: 0.9900 - val_loss: 0.0492 - val_accuracy: 0.9800
Epoch 469/500
2/2 [==============================] - 0s 48ms/step - loss: 0.0273 - accuracy: 0.9900 - val_loss: 0.0584 - val_accuracy: 0.9800
Epoch 470/500
2/2 [==============================] - 0s 46ms/step - loss: 0.0305 - accuracy: 0.9700 - val_loss: 0.0641 - val_accuracy: 0.9800
Epoch 471/500
2/2 [==============================] - 0s 48ms/step - loss: 0.0340 - accuracy: 0.9800 - val_loss: 0.0460 - val_accuracy: 0.9800
Epoch 

<h3 align='center'> 4.1 Testing </h3>
 
We are going to resue the previous code from 3.1 and test our new model's working we are going to get max output of multi label classifier which will essentially predict similar to multi class classification 

In [9]:
##### input_1=[5.1,3.5,1.4,0.2]
print(predict_output(multi_label_model,input_1))

input_2=get_input_from_users(dataset_label_encoded.shape[1]-1)
print(predict_output(multi_label_model,input_2))

1/1 [==============================] - 0s 131ms/step
[[7.5012031e-03 2.7883414e-05 9.9711472e-01]]
Iris-setosa
Enter 1 ipnut 1.5
Enter 2 ipnut 3.5
Enter 3 ipnut 5.5
Enter 4 ipnut 4.4
1/1 [==============================] - 0s 67ms/step
[[5.512478e-03 9.972457e-01 6.086845e-05]]
Iris-virginica
